# Grid Data Extraction

We will extract the DEM's for each grid in the filtered grid shapefile.

1. Open the ArcticDEM Strip Shapefile
2. Open the filtered grid shapefile
3. Filter the Strip shapefile to only include DEMs which intersect the grids (this reduces the Strip count to 378)

## Imports

In [ ]:
!pip install pandarallel --user

In [11]:
# GIS
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
import json

# Multiprocessing 
from functools import partial
from multiprocessing import Pool
from pandarallel import pandarallel

# General Use
import os
import pandas as pd
import numpy as np
import glob

# Opening files which fail to open
import tarfile
import gzip
import urllib

## Create a new index

We will create an index of the DEM strips using this new filtered shapefile.

In [3]:
if os.path.exists('./data/index.pkl'):
    index = pd.read_pickle('./data/index.pkl')
    print('Read index from file')
else:
    print('Creating new index from shapefile.')
    index = gpd.read_file('../../data/Filtered_ArcticDEM_Strip_Index_Rel7/ArcticDEM_Strip_Index_Rel7.shp')
    index = index.set_index('name', drop=True)

Read index from file


### Select only useful columns
#### Note that the acquisitio and acquisit_1 columns differ by a few days, affecting 80 strips. I will use the acquisito column.

In [10]:
dates = index[['acquisitio', 'acquisit_1']]
diff = dates['acquisitio'] == dates['acquisit_1']
diff = diff[diff == False]
diff = index.loc[diff.index]
print('Rasters affected:', len(diff))
diff.head()

Rasters affected: 80


,pairname,catalogid1,catalogid2,nd_value,resolution,algm_ver,creationda,raster,fileurl,acquisitio,...,num_gcps,meanresz,active,qc,rel_ver,acquisit_1,sensor2,st_area_sh,st_length_,geometry
name,,,,,,,,,,,,,,,,,,,,,
SETSM_W1W1_20150616_1020010040333000_1020010041CFE600_seg2_2m_v3.0,W1W1_20150616_1020010040333000_1020010041CFE600,1020010040333000,1020010041CFE600,-9999.0,2.0,SETSM 3.2.8,2017-12-26,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,2015-06-20,...,0,0.000,1,0,7,2015-06-16,WV01,9.102732e+07,54719.937383,"POLYGON ((-2074088.000 769466.000, -2074048.00..."
SETSM_W1W2_20120316_1020010018A84900_1030010012990F00_seg1_2m_v3.0,W1W2_20120316_1020010018A84900_1030010012990F00,1020010018A84900,1030010012990F00,-9999.0,2.0,SETSM 3.2.8,2017-12-26,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,2012-03-25,...,1021,-0.022,1,2,7,2012-03-16,WV02,1.050584e+09,243817.056635,"POLYGON ((-2046828.000 777020.000, -2042598.00..."
SETSM_W1W2_20160720_1020010053AB7100_10300100570FB000_seg1_2m_v3.0,W1W2_20160720_1020010053AB7100_10300100570FB000,1020010053AB7100,10300100570FB000,-9999.0,2.0,SETSM 3.2.8,2017-12-26,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,2016-07-20,...,545,-0.005,1,0,7,2016-07-24,WV02,1.132057e+09,188763.283589,"POLYGON ((-2050624.000 767014.000, -2050520.00..."
SETSM_W1W2_20150404_102001003D671800_1030010040656F00_seg1_2m_v3.0,W1W2_20150404_102001003D671800_1030010040656F00,102001003D671800,1030010040656F00,-9999.0,2.0,SETSM 3.2.8,2017-12-26,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,2015-04-04,...,526,-0.005,1,0,7,2015-04-06,WV02,1.864671e+08,88050.575563,"POLYGON ((-2013562.000 747518.000, -2013546.00..."
SETSM_W1W1_20100321_102001000CC6E100_102001000CE89900_seg1_2m_v3.0,W1W1_20100321_102001000CC6E100_102001000CE89900,102001000CC6E100,102001000CE89900,-9999.0,2.0,SETSM 3.2.8,2017-12-26,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,2010-03-21,...,2361,-0.003,1,0,7,2010-03-25,WV01,1.966518e+09,254475.760197,"POLYGON ((-2058326.000 742190.000, -2058086.00..."


In [11]:
index.columns

Index(['pairname', 'catalogid1', 'catalogid2', 'nd_value', 'resolution',
       'algm_ver', 'creationda', 'raster', 'fileurl', 'acquisitio',
       'spec_type', 'sensor1', 'qual', 'dx', 'dy', 'dz', 'reg_src', 'num_gcps',
       'meanresz', 'active', 'qc', 'rel_ver', 'acquisit_1', 'sensor2',
       'st_area_sh', 'st_length_', 'geometry'],
      dtype='object')

In [12]:
index = index[['acquisitio', 'fileurl', 'dx', 'dy', 'dz', 'geometry']]

In [13]:
index.head()

,acquisitio,fileurl,dx,dy,dz,geometry
name,,,,,,
SETSM_GE01_20120812_10504100007CE100_1050410000778700_seg1_2m_v3.0,2012-08-12,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,0.188,-0.399,-0.867,"POLYGON ((-2087418.000 669438.000, -2087130.00..."
SETSM_GE01_20120812_10504100007A7300_1050410000751500_seg1_2m_v3.0,2012-08-12,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,-0.980,-0.374,1.334,"POLYGON ((-2086408.000 672442.000, -2086400.00..."
SETSM_GE01_20120813_1050410000870B00_1050410000847E00_seg1_2m_v3.0,2012-08-13,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,-1.759,-2.802,0.454,"POLYGON ((-2087612.000 670136.000, -2087604.00..."
SETSM_WV01_20130411_1020010022450500_1020010021AB8500_seg1_2m_v3.0,2013-04-11,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,-0.496,-0.160,-0.306,"POLYGON ((-2168618.000 787874.000, -2168594.00..."
SETSM_WV01_20140708_102001002F474A00_102001003026E300_seg1_2m_v3.0,2014-07-08,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,-0.140,0.114,-1.551,"POLYGON ((-2162648.000 786644.000, -2161808.00..."


In [12]:
index.to_pickle('./data/index.pkl')

## Load Grid Shapefile

In [4]:
grids = gpd.read_file('../../data/grid_shapefile/1km_filtered/filtered.shp')
grids['id'] = grids['id'].astype(int)
grids = grids.set_index('id', drop=True)

## Extract Each Grid's Data

We will iterate over the Strip frame, find each grid that it intersects, and extract the data for those grids.

In [5]:
def find_grid_intersections(raster):
    '''
    Given a row (Strip raster) of the index, this function returns a list of the grids it intersects.
    '''
    
    intersection = []
    for _, grid in grids.iterrows():
        if grid['geometry'].intersects(raster['geometry']):
            intersection.append(grid)
    return intersection

In [6]:
def mask_grids(raster, overwrite=False):
    '''
    Given a row (Strip raster) of the index, this function downloads all of the rasters for the grids it intersects.
    
    If there is 100% No Data, the file is marked as raster_name + '_NODATA_dem.tif'
    If the raster file is failed to open by rasterio, the file is marked as raster_name + '.txt'
    '''
    rio_url = 'tar+' + raster['fileurl'] + '!' + raster.name + '_dem.tif'
    try:  
        src = rio.open(rio_url)
    except:  # RasterIO fails to open the URL, mark that raster for a second processing
        with open('./data/grids/' + raster.name + '.txt', 'w') as dst:
            pass
        src = open_dem(raster)
    

    for grid in find_grid_intersections(raster):
        out_dir = './data/grids/' + str(grid.name) + '/'
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        # Convert Shapely geometry to GeoJSON
        geo = gpd.GeoDataFrame({'geometry': grid['geometry']}, index=[0], crs=src.crs)
        geo = [json.loads(geo.to_json())['features'][0]['geometry']]

        out_img, out_transform = mask(src, shapes=geo, crop=True)  # Mask and crop the raster to the grid
        out_img = np.squeeze(out_img)  # Reduce shape to 2D (used to be (1, width, height))

        # Update the TIF metadata
        out_meta = src.meta.copy()
        out_meta.update({'driver':'GTiff',
                         'height': out_img.shape[0],
                         'width': out_img.shape[1],
                         'transform': out_transform,
                         'crs': src.crs
                        })

        # Check for 0% Data. 
        msk = np.ma.masked_equal(out_img, src.nodata)
        if np.all(msk.mask):  # If all of the values are True (no data), mark the file as having no data.
            outfile = out_dir + raster.name + '_NODATA_dem.tif'
        else:
            outfile = out_dir + raster.name + '_dem.tif'

        # Write to outfile
        with rio.open(outfile, 'w', **out_meta) as dst:
            dst.write(out_img, 1)
    return

## A Second Processing

Some of the rasters failed to open by RasterIO, they are marked as RASTER_NAME + '.txt' in the data directory.

We will process them again with a different method of loading.


#### Sample Error Message
RasterioIOError: '/vsitar/vsicurl/http://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/geocell/v3.0/2m/n69w156/SETSM_WV01_20130411_1020010022450500_1020010021AB8500_seg1_2m_v3.0.tar.gz/SETSM_WV01_20130411_1020010022450500_1020010021AB8500_seg1_2m_v3.0_dem.tif' not recognized as a supported file format.

In [7]:
failed = glob.glob('./data/grids/*.txt')
failed = [x[x.rfind('/')+1:x.rfind('.')] for x in failed]
print(len(failed))
fail_index = index.loc[failed]

38


In [29]:
def open_dem(raster):
    tempfile = urllib.request.urlretrieve(raster['fileurl'], filename=None)[0]
    
    tar = tarfile.open(tempfile)
    tar.extract(raster.name + '_dem.tif')
    src = rio.open('./' + raster.name + '_dem.tif')
    os.remove('./' + raster.name + '_dem.tif')
    return src

In [31]:
for _, raster in fail_index.iterrows():
    print(raster.name)
    
    grids = []
    for grid in find_grid_intersections(raster):
        out_dir = './data/grids/' + str(grid.name) + '/'
        if not os.path.exists(out_dir + raster.name + '_NODATA_dem.tif') and not os.path.exists(out_dir + raster.name + '_dem.tif'):
            grids.append(grid)
    
    src = open_dem(raster)
    
    for grid in grids:
        out_dir = './data/grids/' + str(grid.name) + '/'
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        # Convert Shapely geometry to GeoJSON
        geo = gpd.GeoDataFrame({'geometry': grid['geometry']}, index=[0], crs=src.crs)
        geo = [json.loads(geo.to_json())['features'][0]['geometry']]

        out_img, out_transform = mask(src, shapes=geo, crop=True)  # Mask and crop the raster to the grid
        out_img = np.squeeze(out_img)  # Reduce shape to 2D (used to be (1, width, height))

        # Update the TIF metadata
        out_meta = src.meta.copy()
        out_meta.update({'driver':'GTiff',
                         'height': out_img.shape[0],
                         'width': out_img.shape[1],
                         'transform': out_transform,
                         'crs': src.crs
                        })

        # Check for 0% Data. 
        msk = np.ma.masked_equal(out_img, src.nodata)
        if np.all(msk.mask):  # If all of the values are True (no data), mark the file as having no data.
            outfile = out_dir + raster.name + '_NODATA_dem.tif'
        else:
            outfile = out_dir + raster.name + '_dem.tif'

        # Write to outfile
        with rio.open(outfile, 'w', **out_meta) as dst:
            dst.write(out_img, 1)

SETSM_WV01_20130411_1020010022450500_1020010021AB8500_seg1_2m_v3.0
SETSM_WV01_20131005_1020010025D7AD00_1020010026768200_seg1_2m_v3.0
SETSM_WV01_20150404_102001003B5A0F00_102001003D671800_seg1_2m_v3.0
SETSM_WV01_20120710_102001001C6F3E00_102001001DC92D00_seg1_2m_v3.0
SETSM_W1W2_20120316_1020010018A84900_1030010012990F00_seg1_2m_v3.0
SETSM_WV01_20170614_1020010061544600_102001005F51B800_seg1_2m_v3.0
SETSM_WV01_20120614_102001001A94E300_102001001B600200_seg1_2m_v3.0
SETSM_WV01_20150314_102001003ADB2E00_102001003C71EC00_seg1_2m_v3.0
SETSM_WV01_20120506_102001001B095100_102001001BE66800_seg1_2m_v3.0
SETSM_WV01_20120524_102001001AC6E100_102001001AB0DE00_seg1_2m_v3.0
SETSM_WV01_20140404_102001002B398C00_102001002E6CE300_seg1_2m_v3.0
SETSM_WV01_20170528_102001005F84FD00_10200100628EA700_seg1_2m_v3.0
SETSM_WV01_20170603_102001005F7D2B00_10200100635E4D00_seg1_2m_v3.0
SETSM_WV01_20120605_102001001B65AA00_102001001AA32500_seg1_2m_v3.0
SETSM_W1W1_20120615_1020010019C06900_102001001A91EC00_seg1_2m_